# LUAD CPTAC analysis

In [2]:
import pandas as pd
import numpy as np
from msresist.pre_processing import preprocessing
from msresist.clustering import MassSpecClustering
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid", font_scale=1, color_codes=True, palette="colorblind", rc={"grid.linestyle": "dotted", "axes.linewidth": 0.6})

In [3]:
# pd.set_option('display.max_colwidth', 1000)
# pd.set_option('display.max_rows', 1000000)
# pd.set_option('display.max_columns', 1000)

## 1 - Import and Preprocess Data

In [4]:
# X = preprocessing(CPTAC=True, log2T=True)
X = pd.read_csv("msresist/data/MS/CPTAC/CPTAC-preprocessedMotfis.csv")

d = X.select_dtypes(include=['float64']).T
i = X.select_dtypes(include=['object'])

In [5]:
X

,Unnamed: 0,Sequence,Protein,Gene,Position,C3N.01799,C3N.01799.N,C3L.01890,C3L.01890.N,C3N.00572,...,C3N.02003.N,C3N.02379.N,C3N.00175,C3N.00175.N,C3N.01823,C3N.01823.N,C3L.02549,C3L.02549.N,C3L.02365,C3L.02365.N
0,0,AAAAAsGGGVS,NP_002577.2,PBX2,S146-p,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,AAAAAsQQGSA,NP_001308122.1,TBL1XR1,S119-p,NaN,NaN,NaN,NaN,NaN,...,-0.372907,-0.979491,-0.083036,-0.361962,1.275506,-0.568015,0.254230,-0.378522,0.987749,-0.189486
2,2,AAAAAsTQAQQ,NP_001035540.1,MINDY2,S575-p,0.012034,-0.630863,-1.159479,-0.740044,0.107228,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,AAAAGsASPRS,NP_064520.2,WRNIP1,S151-p,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,AAAAGsGPSPP,NP_060895.1,PI4K2A,S44-p,NaN,NaN,NaN,NaN,NaN,...,-0.063300,-0.647500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42378,42378,YYSTEsEPLTN,NP_055869.1,UHRF1BP1L,S777-p,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42379,42379,YYTAGyNSPVK,NP_001313309.1,HEBP2,Y179-p,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-0.631923,-1.911083,0.620938,-0.654537,0.974769,-1.261869,0.726954,-1.583683
42380,42380,YYTIPsMDDLA,NP_057404.2,NUP98,S746-p,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42381,42381,YYTSAsGDEMV,NP_001017963.2,HSP90AA1,S470-p,NaN,NaN,NaN,NaN,NaN,...,-0.719128,-0.937400,0.296906,-0.821107,0.837424,-0.427417,0.716933,-0.880607,0.606949,-0.533247


## 2 - Run Model

Test speed of generating the motif pam250 scores:

In [6]:
# %%time
# from msresist.sequence_analysis import MotifPam250Scores
# n = 5000
# seqs = [s.upper() for s in X["Sequence"][:n]]
# print(len(seqs))
# MotifPam250Scores(seqs)

Run model:

In [7]:
# Reduce data set?
npept = 7000
d = d.iloc[:, :npept]
i = i.iloc[:npept, :]

#Drop NaN?
# d = d.iloc[:8, :].dropna(axis=1)
# i = i.iloc[d.columns, :]

In [ ]:
%%time
distance_method = "PAM250"
ncl = 2
SeqWeight = 0

MSC = MassSpecClustering(i, ncl, GMMweight=SeqWeight, distance_method=distance_method, n_runs=1).fit(d, "NA")

In [ ]:
# centers = MSC.transform(d)

In [ ]:
from msresist.figures.figure1 import plotClustergram
import seaborn as sns
plotClustergram(centers.T, "", lim=max(np.max(np.abs(centers.T)) * 0.2), figsize=(20, 20))
# plt.savefig("CPTAC_GMM.svg", bbox_inches='tight', format='svg', dpi=1200)

In [ ]:
# fig, ax = plt.subplots(1, 1, figsize=(7,6))
# p1 = PLSRegression(n_components=2)
# plotR2YQ2Y(ax, p1, d, y_, 1, 10)

In [ ]:
cf = pd.read_csv("Y_CPTAC.csv")
cf_ = cf.replace("Living", 0)
cf_ = cf_.replace("Deceased", 1)
cf_ = cf_.replace("No", 0)
cf_ = cf_.replace("Yes", 1)

In [ ]:
IDict = pd.read_csv("IDs.csv", header=0)
IDict_ = dict(zip(IDict.iloc[:, 0], IDict.iloc[:, 1]))

In [ ]:
ids = []
for i in range(cf.shape[0]):
    ids.append(IDict_[cf_.iloc[i, 0]])
cf_["Patient_ID"] = ids

In [ ]:
for i in range(cf_.shape[0]):
    id_ = cf_.iloc[i, 0].split(".N")[0]
    cf_.loc[-1] = [id_, cf_.iloc[i, 1], cf_.iloc[i, 2]]
    cf_.index = cf_.index + 1
cf_.loc[-1] = ["C3N.02379.1", 0, 0]
cf_.index = cf_.index + 1
cf_ = cf_.sort_index()

In [ ]:
for col in centers.columns:
    if col not in list(cf_["Patient_ID"]):
        print(col)

In [ ]:
centers = centers.drop(["C3N.02587", "C3N.02587.N"], axis=1)

In [ ]:
y = pd.read_csv("y.csv").set_index("Patient_ID")

In [ ]:
centers = centers.reset_index().drop("Clusters", axis=1).T.reset_index().sort_values(by="index")
y = y.reset_index().sort_values(by="Patient_ID")

In [ ]:
centers = centers.set_index("index")
y = y.set_index("Patient_ID")

In [ ]:
from msresist.figures.figure3 import plotR2YQ2Y
fig, ax = plt.subplots(1, 1, figsize=(7,6))
p1 = PLSRegression(n_components=2, scale=False)
plotR2YQ2Y(ax, p1, centers, y, 1, 10)

In [ ]:
centers.head()

In [ ]:
list(range(20))

In [ ]:
centers = centers.drop("C3N.00738.N")

In [ ]:
centers = centers.reset_index()

In [ ]:
from sklearn.decomposition import PCA
from msresist.figures.figure1 import pca_dfs

c=2
pp = PCA(n_components=c)
dScor_ = pp.fit_transform(centers.iloc[:, 1:])
dLoad_ = pp.components_
dScor_, dLoad_ = pca_dfs(dScor_, dLoad_, centers, c, ["index"], "Cluster")

In [ ]:
dScor_.columns = ["PC1", "PC2", "Patient_ID"]

In [ ]:
tumortype = []
for i in range(dScor_.shape[0]):
    if ".N" in dScor_["Patient_ID"][i]:
        tumortype.append("Normal")
    else:
        tumortype.append("Tumor")

In [ ]:
dScor_["Type"] = tumortype

In [ ]:
varExp = np.round(pp.explained_variance_ratio_, 2)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(7, 7))
sns.scatterplot(x="PC1", y="PC2", data=dScor_, hue="Type", style="Type", ax=ax, s=80, **{'linewidth':.5, 'edgecolor':"k"})
ax.set_title("PCA Scores", fontsize=15)
ax.set_xlabel("PC1 (" + str(int(varExp[0] * 100)) + "%)", fontsize=14)
ax.set_ylabel("PC2 (" + str(int(varExp[1] * 100)) + "%)", fontsize=14)
ax.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0, labelspacing=0.2, fontsize=15);
plt.savefig("PCA_CPTAC_20cl_Scores.svg", bbox_inches='tight', format='svg', dpi=1200)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(7, 7))
sns.scatterplot(x="PC1", y="PC2", data=dLoad_, hue="Cluster", ax=ax, s=80, **{'linewidth':.5, 'edgecolor':"k"})
for line in range(0, dLoad_.shape[0]):
     ax.text(dLoad_.iloc[line, 0]+0.02, dLoad_.iloc[line, 1]+0.02, dLoad_["Cluster"][line], horizontalalignment='right', size='medium', color='black', weight='semibold')
ax.legend().remove()
ax.set_title("PCA Loadings", fontsize=15)
ax.set_xlabel("PC1 (" + str(int(varExp[0] * 100)) + "%)", fontsize=14)
ax.set_ylabel("PC2 (" + str(int(varExp[1] * 100)) + "%)", fontsize=14);
plt.savefig("PCA_CPTAC_20cl_Loadings.svg", bbox_inches='tight', format='svg', dpi=1200)